In [130]:
from datetime import datetime 
from pony.orm import *
from dash import *
import pandas as pd
import json
import os

In [ ]:
exercise_data = open(f'/Users/Golna/Documents/Health_Care/Health_Care/pmdata/p01/fitbit/exercise.json')
sleep_data = open(f'/Users/Golna/Documents/Health_Care/Health_Care/pmdata/p01/fitbit/sleep.json')
calories_data = open(f'/Users/Golna/Documents/Health_Care/Health_Care/pmdata/p01/fitbit/calories.json')
distance_data = open(f'/Users/Golna/Documents/Health_Care/Health_Care/pmdata/p01/fitbit/distance.json')
sleep_score_data = pd.read_csv('/Users/Golna/Documents/Health_Care/Health_Care/pmdata/p01/fitbit/sleep_score.csv')
partecipant_data = pd.read_excel('/Users/Golna/Documents/Health_Care/Health_Care/pmdata/participant-overview.xlsx')


In [99]:
e = json.load(exercise_data)
s = json.load(sleep_data)
c = json.load(calories_data)
d = json.load(distance_data)

In [100]:
participant_list = partecipant_data.iloc[0].tolist()
SleepScore_list = sleep_score_data.values.tolist()
exercise_list = list(e)
Sleep_list = list(s)
Calories_list = list(c)
Distance_list = list(d)

In [101]:
#participant_list

In [102]:
db = Database('sqlite', os.getcwd()+ '/Final.sqlite', create_db = True)

class Participant(db.Entity):
    id = PrimaryKey(str)
    age = Optional(int)
    height = Optional(int)
    gender = Optional(str)
    calories = Set("Calories")
    exercise = Set("Exercise")
    sleep = Set("Sleep")
    distance = Set("Distance")
    sleep_score = Set("Sleep_score")
    exercise_Fact = Set("Exercise_Fact")
    daily_activity_fact = Set("Daily_Activity_Fact")
    
    
class Calories (db.Entity):
    valid_time = Required(datetime)
    participant = Required(Participant)
    value = Optional(float)
    PrimaryKey(valid_time, participant)


class Exercise (db.Entity):
    valid_time = Required(datetime)
    participant = Required(Participant)
    calories = Optional(int)
    #logId = Required(int)
    duration = Optional(int) # in minutes
    PrimaryKey(valid_time, participant)

class Sleep (db.Entity):
    vt = Required(datetime)
    participant = Required(Participant)
    duration = Optional(int)
    PrimaryKey(vt, participant)

class Distance (db.Entity):
    valid_time = Required(datetime)
    participant = Required(Participant)
    value = Optional(float) 
    PrimaryKey(valid_time, participant)

class Sleep_score (db.Entity):
    valid_time = Required(datetime)
    participant = Required(Participant)
    duration = Optional(int)
    PrimaryKey(valid_time, participant)

class Exercise_Fact(db.Entity):
    participant = Required(Participant)
    valid_time = Required(datetime)
    total_calories = Optional(float)
    exercise_count = Optional(int)
    Gender = Optional(str)
    Age = Optional(int)
    Duration = Optional(int)
    
    PrimaryKey(participant, valid_time)
    
class Daily_Activity_Fact(db.Entity):
    participant = Required(Participant)
    valid_time = Required(datetime)
    exercise_duration = Optional(int)
    calories = Optional(float)
    sleep_duration = Optional(float)
    distance_cover = Optional(float)
    PrimaryKey(valid_time, participant)



In [103]:
db.generate_mapping(create_tables=True)

In [104]:
#partecipant_data

In [105]:
# Assuming 'df' is your DataFrame
#first_row = partecipant_data.iloc[[0]]

In [106]:
#first_row 

In [107]:
def load_participant(df):
    with db_session:
        for index , row in df.iterrows():
            Participant(
                id = row['Participant_ID'],
                age = row['Age'],
                height = row['Height'],
                gender = row['Gender']
            )

In [108]:
load_participant(partecipant_data)

In [109]:
db.commit()

In [110]:
@db_session
def load_exercise_data(id, exercise_data_path):
    if os.path.isfile(exercise_data_path):
        exercise_data = pd.read_csv(exercise_data_path)
        
        # Convert the 'startTime' column to datetime
        exercise_data['startTime'] = pd.to_datetime(exercise_data['startTime'])

        for row in exercise_data.itertuples(index=False):  # itertuples is more efficient than iterrows
            Exercise(
                participant=id,
                valid_time=row.startTime,
                duration=row.originalDuration,
                calories = row.calories
            )



In [111]:
@db_session
def load_calories_data(id, calories_data_path):
    if os.path.isfile(calories_data_path):
        calories_data = pd.read_csv(calories_data_path)
        
        # Convert the 'startTime' column to datetime
        calories_data['dateTime'] = pd.to_datetime(calories_data['dateTime'])

        for row in calories_data.itertuples(index=False):  # itertuples is more efficient than iterrows
            Calories(
                participant=id,
                valid_time=row.dateTime,
                value=row.value
            )



In [112]:
@db_session
def load_sleep_data(id, sleep_data_path):
    if os.path.isfile(sleep_data_path):
        sleep_data = pd.read_csv(sleep_data_path)
        
        # Convert the 'startTime' column to datetime
        sleep_data['startTime'] = pd.to_datetime(sleep_data['startTime'])

        for row in sleep_data.itertuples(index=False):
            # Check if the record already exists
            existing_record = Sleep.get(participant=id, vt=row.startTime)

            if existing_record:
                print(f"Record for participant {id} at {row.startTime} already exists. Skipping...")
                continue

            # Create a new Sleep record
            Sleep(
                participant=id,
                vt=row.startTime,
                duration=row.duration
            )
        commit()  # Commit changes after processing all records


In [113]:
@db_session
def load_distance_data(id, distance_data_path):
    if os.path.isfile(distance_data_path):
        distance_data = pd.read_csv(distance_data_path)
        
        # Convert the 'startTime' column to datetime
        distance_data['dateTime'] = pd.to_datetime(distance_data['dateTime'])

        for row in distance_data.itertuples(index=False):
            # Check if the record already exists
            existing_record = Distance.get(participant=id, valid_time=row.dateTime)

            if existing_record:
                print(f"Record for participant {id} at {row.dateTime} already exists. Skipping...")
                continue

            # Create a new Distance record
            Distance(
                participant=id,
                valid_time=row.dateTime,
                value=row.value
            )
        commit()  # Commit changes after processing all records


In [114]:
@db_session
def load_sleep_score_data(id, sleep_score_data_path):
    if os.path.isfile(sleep_score_data_path):
        sleep_score_data = pd.read_csv(sleep_score_data_path)
        
        # Convert the 'startTime' column to datetime
        sleep_score_data['timestamp'] = pd.to_datetime(sleep_score_data['timestamp'])

        for row in sleep_score_data.itertuples(index=False):  # itertuples is more efficient than iterrows
            Sleep_score(
                participant=id,
                valid_time=row.timestamp,
                duration=row.duration_score
            )



In [115]:
# Try to oplimize the code that use for loadind data to database But it dosen't work!
#I don't know why?
import concurrent.futures

# Define a function to load data for a single participant
def load_data_for_participant(participant):
    participant_id = f"p{participant:02d}"
    base_path = "/Users/majidtavakoli/Documents/Health_Care/Health_Care/CSV_Data/"

    # Exercise data:
    exercise_path = base_path + f"exercise{participant_id}.csv"
    load_exercise_data(participant_id, exercise_path)

    # Calories data:
    calories_path = base_path + f"calories{participant_id}.csv"
    load_calories_data(participant_id, calories_path)

    # Sleep data:
    sleep_path = base_path + "sleep01.csv"  # Assuming consistent path for all participants
    load_sleep_data(participant_id, sleep_path)

    # Distance data (loaded once):
    distance_path = base_path + f"distance{participant_id}.csv"
    load_distance_data(participant_id, distance_path)

    # Sleep score data:
    sleep_score_path = base_path + f"sleep_score{participant_id}.csv"
    load_sleep_score_data(participant_id, sleep_score_path)

# Use ThreadPoolExecutor to load data for multiple participants concurrently
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Load data for participants 1 to 5 concurrently
    participant_numbers = range(1, 6)
    executor.map(load_data_for_participant, participant_numbers)


Record for participant p01 at 2019-11-06 23:45:30 already exists. Skipping...
Record for participant p01 at 2019-12-06 23:29:00 already exists. Skipping...
Record for participant p01 at 2020-01-06 00:35:30 already exists. Skipping...
Record for participant p01 at 2020-02-05 00:16:30 already exists. Skipping...
Record for participant p01 at 2020-03-05 23:45:00 already exists. Skipping...
Record for participant p03 at 2019-11-06 23:45:30 already exists. Skipping...
Record for participant p03 at 2019-12-06 23:29:00 already exists. Skipping...
Record for participant p03 at 2020-01-06 00:35:30 already exists. Skipping...
Record for participant p03 at 2020-02-05 00:16:30 already exists. Skipping...
Record for participant p03 at 2020-03-05 23:45:00 already exists. Skipping...
Record for participant p05 at 2019-11-06 23:45:30 already exists. Skipping...
Record for participant p05 at 2019-12-06 23:29:00 already exists. Skipping...
Record for participant p05 at 2020-01-06 00:35:30 already exists

In [ ]:
for participant_number in range(1, 6):
    participant = f"{participant_number:02d}"

    # Exercise data:
    path = r"/Users/majidtavakoli/Documents/Health_Care/Health_Care/CSV_Data/exercise" + participant + r".csv"
    load_exercise_data("p" + participant, path)

    # Calories data:
    path = r"/Users/majidtavakoli/Documents/Health_Care/Health_Care/CSV_Data/calories" + participant + r".csv"
    load_calories_data("p" + participant, path)

    # Sleep data (using a fixed path for "sleep01.csv"):
    path = r"/Users/majidtavakoli/Documents/Health_Care/Health_Care/CSV_Data/sleep01.csv"  # Assuming consistent path for all participants
    load_sleep_data("p" + participant, path)

    # Distance data (loading once):
    path = r"/Users/majidtavakoli/Documents/Health_Care/Health_Care/CSV_Data/distance" + participant + r".csv"
    load_distance_data("p" + participant, path)

    # Sleep score data:
    path = r"/Users/majidtavakoli/Documents/Health_Care/Health_Care/CSV_Data/sleep_score" + participant + r".csv"
    load_sleep_score_data("p" + participant, path)


In [117]:
db.commit()

In [118]:
from datetime import date

def populate_exercise_fact():
    for participant in Participant.select():
        for exercise in participant.exercise:
            exercise_date = exercise.valid_time
            # Check if an entry for the participant and date already exists
            existing_entry = Exercise_Fact.get(participant=participant, valid_time=exercise_date)
            
            
            Exercise_Fact(
                participant=participant,
                valid_time=exercise_date,
                total_calories=exercise.calories,
                exercise_count=1,
                Gender = participant.gender,
                Age = participant.age,
                Duration = exercise.duration
                )
    commit()

populate_exercise_fact()

In [119]:
@db_session
def populate_daily_activity_fact():
    participants = Participant.select()
    for participant in participants:
        # Retrieve exercise duration for the participant
        exercise_duration = sum(exercise.duration for exercise in participant.exercise)
        
        # Retrieve total calories for the participant
        total_calories = sum(calories.value for calories in participant.calories)
        
        # Retrieve sleep duration for the participant
        sleep_duration = sum(sleep.duration for sleep in participant.sleep)
        sleep_duration_str = str(sleep_duration)
        # Calculate distance covered if applicable
        # (You need to define a mechanism to calculate distance covered based on your data)
        #distance_covered = calculate_distance_covered(participant)
        
        # Create Daily_Activity_Fact record
        Daily_Activity_Fact(
            participant=participant,
            valid_time=datetime.now(),  # Replace with appropriate date/time
            exercise_duration=exercise_duration,
            calories=total_calories,
            sleep_duration=sleep_duration_str,
            #distance_covered=distance_covered
            # Add other attributes as needed
        )

In [120]:
populate_daily_activity_fact()

In [121]:
db.commit()

In [ ]:
query = select((f.participant.id, f.valid_time.date(), f.exercise_duration, f.calories, f.sleep_duration)
               for f in Daily_Activity_Fact)
fact_data = query[:]


In [131]:


# Assume you have already retrieved data from your database and stored it in a DataFrame
# For demonstration purposes, let's create a sample DataFrame
data = {
    'Date': pd.date_range(start='2022-01-01', end='2022-01-31'),
    'Exercise_Duration': [60, 45, 70, 80, 90, 75, 60, 50, 45, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165],
    'Calories_Burned': [200, 180, 220, 250, 270, 230, 200, 190, 180, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400,410]
}
df = pd.DataFrame(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Fitness Dashboard"),
    
    dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=df['Date'].min(),
        max_date_allowed=df['Date'].max(),
        start_date=df['Date'].min(),
        end_date=df['Date'].max(),
        display_format='YYYY-MM-DD',
    ),
    
    dcc.Graph(id='exercise-duration-graph'),
    dcc.Graph(id='calories-burned-graph')
])

# Define callback functions to update the graphs based on user input
@app.callback(
    Output('exercise-duration-graph', 'figure'),
    Output('calories-burned-graph', 'figure'),
    Input('date-picker-range', 'start_date'),
    Input('date-picker-range', 'end_date')
)
def update_graphs(start_date, end_date):
    filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    # Create a line chart for exercise duration
    exercise_duration_fig = px.line(filtered_df, x='Date', y='Exercise_Duration', title='Exercise Duration over Time')
    
    # Create a line chart for calories burned
    calories_burned_fig = px.line(filtered_df, x='Date', y='Calories_Burned', title='Calories Burned over Time')
    
    return exercise_duration_fig, calories_burned_fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


In [129]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px

# Sample data
data = {
    'Category': ['A', 'B', 'C', 'D', 'E'],
    'Value': [10, 20, 15, 25, 30]
}

# Create DataFrame
df = pd.DataFrame(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div(children=[
    html.H1(children='Simple Dashboard Example'),

    html.Div(children='''
        A simple example of a dashboard using Dash and Plotly.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=px.bar(df, x='Category', y='Value', title='Sample Bar Chart')
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
